In [83]:
import numpy as np
import pandas as pd

In [95]:
df = pd.read_csv('hacktrain.csv')
df.head()

,Unnamed: 0,ID,class,20150720_N,20150602_N,20150517_N,20150501_N,20150415_N,20150330_N,20150314_N,...,20140610_N,20140525_N,20140509_N,20140423_N,20140407_N,20140322_N,20140218_N,20140202_N,20140117_N,20140101_N
0,0,1,water,637.5950,658.668,-1882.030,-1924.36,997.904,-1739.990,630.087,...,NaN,-1043.160,-1942.490,267.138,NaN,NaN,211.328,-2203.020,-1180.19,433.906
1,1,2,water,634.2400,593.705,-1625.790,-1672.32,914.198,-692.386,707.626,...,NaN,-933.934,-625.385,120.059,364.858,476.972,220.878,-2250.000,-1360.56,524.075
2,3,4,water,58.0174,-1599.160,NaN,-1052.63,NaN,-1564.630,NaN,...,-1025.88,368.622,NaN,-1227.800,304.621,NaN,369.214,-2202.120,NaN,-1343.550
3,4,5,water,72.5180,NaN,380.436,-1256.93,515.805,-1413.180,-802.942,...,-1813.95,155.624,NaN,-924.073,432.150,282.833,298.320,-2197.360,NaN,-826.727
4,7,8,water,1136.4400,NaN,NaN,1647.83,1935.800,NaN,2158.980,...,1535.00,1959.430,-279.317,-384.915,-113.406,1020.720,1660.650,-116.801,-568.05,-1357.140


In [96]:
df.isnull().sum()

Unnamed: 0       0
ID               0
class            0
20150720_N     560
20150602_N    1200
20150517_N     800
20150501_N     960
20150415_N     480
20150330_N    1120
20150314_N     720
20150226_N    1360
20150210_N     640
20150125_N    1040
20150109_N     880
20141117_N    1280
20141101_N     400
20141016_N    1440
20140930_N     800
20140813_N     560
20140626_N    1600
20140610_N     480
20140525_N     720
20140509_N     880
20140423_N    1760
20140407_N     640
20140322_N    1120
20140218_N    1440
20140202_N     560
20140117_N    1200
20140101_N     400
dtype: int64

In [97]:
df.fillna(df.mean(numeric_only=True), inplace=True)
df.isnull().sum()

Unnamed: 0    0
ID            0
class         0
20150720_N    0
20150602_N    0
20150517_N    0
20150501_N    0
20150415_N    0
20150330_N    0
20150314_N    0
20150226_N    0
20150210_N    0
20150125_N    0
20150109_N    0
20141117_N    0
20141101_N    0
20141016_N    0
20140930_N    0
20140813_N    0
20140626_N    0
20140610_N    0
20140525_N    0
20140509_N    0
20140423_N    0
20140407_N    0
20140322_N    0
20140218_N    0
20140202_N    0
20140117_N    0
20140101_N    0
dtype: int64

In [92]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report

In [98]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
import numpy as np

# Drop ID column
df.drop(columns=['ID'], inplace=True)

# Encode class column
label_encoder = LabelEncoder()
df['class'] = label_encoder.fit_transform(df['class'])

# Feature engineering: create statistical summaries of the time series columns
ndvi_cols = [col for col in df.columns if '_N' in col]  # assuming all NDVI columns end with _N

# Add mean, std, min, max NDVI as new features
df['ndvi_mean'] = df[ndvi_cols].mean(axis=1)
df['ndvi_std'] = df[ndvi_cols].std(axis=1)
df['ndvi_min'] = df[ndvi_cols].min(axis=1)
df['ndvi_max'] = df[ndvi_cols].max(axis=1)

# Fill missing values with column means (simple imputation)
df[ndvi_cols] = df[ndvi_cols].fillna(df[ndvi_cols].mean())

# Create seasonal averages: e.g., average of early/mid/late year NDVI
df['ndvi_early'] = df[ndvi_cols[:9]].mean(axis=1)  # roughly Jan–April
df['ndvi_mid'] = df[ndvi_cols[9:18]].mean(axis=1)  # May–Aug
df['ndvi_late'] = df[ndvi_cols[18:]].mean(axis=1)  # Sep–Dec

# Split features and target
X = df.drop(columns=['class'])
y = df['class']

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# Grid search to find best regularization strength
params = {'C': [0.01, 0.1, 1, 10, 100]}
grid = GridSearchCV(LogisticRegression(max_iter=500, multi_class='multinomial', solver='lbfgs'),
                    params, cv=5)
grid.fit(X_train, y_train)

best_model = grid.best_estimator_

# Predict on test set
y_pred = best_model.predict(X_test)

# Classification report
print("Best C value:", grid.best_params_['C'])
print(classification_report(
    y_test,
    y_pred,
    labels=list(range(len(label_encoder.classes_))),
    target_names=label_encoder.classes_
))


c:\Users\kalpa\OneDrive\Desktop\ML\Kaggle\Kaggle1\kaggle-1st-compitition-\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1264: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(
c:\Users\kalpa\OneDrive\Desktop\ML\Kaggle\Kaggle1\kaggle-1st-compitition-\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 500 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=500).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\kalpa

Best C value: 100
              precision    recall  f1-score   support

        farm       0.86      0.85      0.85       168
      forest       0.99      0.99      0.99      1232
       grass       0.87      0.67      0.75        39
  impervious       0.82      0.90      0.86       134
     orchard       0.29      0.33      0.31         6
       water       0.84      0.76      0.80        21

    accuracy                           0.95      1600
   macro avg       0.78      0.75      0.76      1600
weighted avg       0.96      0.95      0.95      1600



c:\Users\kalpa\OneDrive\Desktop\ML\Kaggle\Kaggle1\kaggle-1st-compitition-\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 500 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=500).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [100]:
from sklearn.metrics import accuracy_score

accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 95.50%


In [113]:
test_data = pd.read_csv("hacktest.csv")
test_data.shape

(2845, 29)

In [114]:
# 1. Impute missing NDVI values
test_data[ndvi_cols] = test_data[ndvi_cols].fillna(test_data[ndvi_cols].mean())

# 2. Feature engineering (same as you did for training)
test_data['ndvi_mean'] = test_data[ndvi_cols].mean(axis=1)
test_data['ndvi_std'] = test_data[ndvi_cols].std(axis=1)
test_data['ndvi_min'] = test_data[ndvi_cols].min(axis=1)
test_data['ndvi_max'] = test_data[ndvi_cols].max(axis=1)
test_data['ndvi_early'] = test_data[ndvi_cols[:9]].mean(axis=1)
test_data['ndvi_mid'] = test_data[ndvi_cols[9:18]].mean(axis=1)
test_data['ndvi_late'] = test_data[ndvi_cols[18:]].mean(axis=1)


In [115]:
ID=test_data['ID']
test_data.drop(['ID'],axis=1,inplace=True)
test_data.fillna(df.mean(numeric_only=True), inplace=True)

In [116]:
y_test = best_model.predict(test_data)

In [117]:
y_test 

array([1, 1, 1, ..., 5, 5, 5], shape=(2845,))

In [118]:
y_decoded = label_encoder.inverse_transform(y_test)
y_decoded

array(['forest', 'forest', 'forest', ..., 'water', 'water', 'water'],
      shape=(2845,), dtype=object)

In [119]:
result = pd.DataFrame({
    'ID': ID,
    'class': y_decoded
})

In [120]:
result

,ID,class
0,1,forest
1,2,forest
2,3,forest
3,4,forest
4,5,forest
...,...,...
2840,2841,water
2841,2842,water
2842,2843,water
2843,2844,water


In [122]:
result.to_csv("submission.csv", index=False)